In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import requests
import tomotools
import configparser
from pprint import pprint

In [ ]:
import subprocess
import nbformat
import os

def _notebook_run(notebook = 'reconstructor-v.1.2.ipynb'):
    """Execute a notebook via nbconvert and collect output.
       :returns (parsed nb object, execution errors)
    """
    
    path = notebook
    args = ["jupyter", "nbconvert", "--execute", "--allow-errors",
            "--ExecutePreprocessor.timeout=-1",
            "--to", "notebook", '--output', notebook, path]
    subprocess.check_call(args)

    args = ["jupyter", "nbconvert", "--to", "html",
            os.path.join(notebook)]
    subprocess.check_call(args)

    nb = nbformat.read(path, nbformat.current_nbformat)
    errors = [output for cell in nb.cells if "outputs" in cell
              for output in cell["outputs"]
              if output.output_type == "error"]
    return nb, errors


In [ ]:
tomo_objects = ['cc0a8c13-34e3-4551-a27f-74224d00b06c',
                ]

In [ ]:
storage_dir = '/diskmnt/a/makov/robotom/'

for to in tomotools.log_progress(tomo_objects):
    tomo_info = tomotools.get_tomoobject_info(to) 
    experiment_id = tomo_info['_id']
    print(tomo_info['specimen'])
    config = configparser.ConfigParser()
    config["SAMPLE"] = tomo_info 
    with open ('tomo.ini', 'w') as cf:
        config.write(cf)
        
    nb, errors = _notebook_run()
    for e in errors:
        pprint(e)
    !cp reconstructor-v.1.2.ipynb {os.path.join(storage_dir, experiment_id)}/ 
    !cp reconstructor-v.1.2.html {os.path.join(storage_dir, experiment_id)}/

In [ ]:
# !git add autotom.ipynb reconstructor.ipynb tomotools.py